In [3]:
import face_recognition
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Cargar el modelo entrenado
modelo = load_model("modelo_rostros.h5", compile=False)

# Definir las etiquetas de las clases
etiquetas = ["Avl", "Cami", "Ichu", "Saul"]

# Cargar la imagen en la que se detectarán rostros
ruta_imagen = "imagen_pruieba.jpg"  # Cambia esto por el path de tu imagen
imagen = cv2.imread(ruta_imagen)
rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

# Detectar las ubicaciones de los rostros
ubicaciones_rostros = face_recognition.face_locations(rgb)

# Si no se encuentran rostros, finalizar
if not ubicaciones_rostros:
    print("No se detectaron rostros en la imagen.")
else:
    # Procesar cada rostro detectado
    for (top, right, bottom, left) in ubicaciones_rostros:
        # Extraer el rostro de la imagen
        rostro = rgb[top:bottom, left:right]

        # Convertir el rostro a escala de grises
        rostro = cv2.cvtColor(rostro, cv2.COLOR_RGB2GRAY)

        # Redimensionar la imagen al tamaño esperado por el modelo
        rostro = cv2.resize(rostro, (224, 224))  # Ajustar al tamaño de entrada del modelo
        rostro = np.expand_dims(rostro, axis=-1)  # Agregar canal para escala de grises
        rostro = np.expand_dims(rostro, axis=0)   # Agregar dimensión para batch
        rostro = rostro / 255.0  # Normalizar valores entre 0 y 1

        # Verificar si la entrada pasa por todo el modelo convolucional
        try:
            prediccion = modelo.predict(rostro)  # El modelo procesa automáticamente la entrada
            clase = np.argmax(prediccion)
            etiqueta = etiquetas[clase]

            # Dibujar la etiqueta y el rectángulo alrededor del rostro
            cv2.rectangle(imagen, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(imagen, etiqueta, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        except Exception as e:
            print(f"Error al clasificar el rostro: {e}")

    # Mostrar la imagen con las detecciones y clasificaciones
    cv2.imshow("Resultados", imagen)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Error al clasificar el rostro: Exception encountered when calling Sequential.call().

Input 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 33856, but received input with shape (1, 186624)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 224, 224, 1), dtype=float32)
  • training=False
  • mask=None
